In [100]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag

import warnings
warnings.filterwarnings("ignore")

In [129]:
# reading in the file #
data=pd.read_excel('C:/Users/wtt09/OneDrive/Desktop/nus proj/ds.xlsx')

In [130]:
data.shape # 100 rows, 36 columns

(100, 36)

In [131]:
data.head() 

,ID,SURVY_START_TIME,SURVY_END_TIME,EMAIL,NAME,HGST_QUA,MAJOR,FRST_JOB_IND,HIRE_DT,DEPRTMT,...,CONFD_EXP_12,SATISIFICATION_IND,CLEANSE_PM,CLEANSE_PT,CLEANSE_PC,CLEANSE_PE,SENTI_SCORE_PM,SENTI_SCORE_PT,SENTI_SCORE_PC,SENTI_SCORE_PE
0,1,2021-08-06 11:42:26,2021-08-06 11:45:19,anonymous,NaN,Bachelor's,Information Technology,No,2021-07-05,NCS Applications,...,Agree,Yes,NaN,NaN,NaN,NaN,0,0,0,0
1,2,2021-08-06 11:42:00,2021-08-06 11:47:17,anonymous,NaN,Bachelor's,Cyber Security,Yes,2020-07-07,NCS Applications,...,Neutral,No,"Understanding, patient, technical","Well-paced, decent, room for learning",Typical government agency,"Varied, full learning, quite enriching",2,2,0,1
2,3,2021-08-06 11:42:36,2021-08-06 11:47:26,anonymous,NaN,Bachelor's,IT,Yes,2019-07-01,NCS Applications,...,Neutral,Yes,Responsibility,"Teamwork, Responsibility","High Security, Teamwork",New Knowledge Learn,0,0,0,0
3,4,2021-08-06 11:44:21,2021-08-06 11:47:47,anonymous,NaN,Bachelor's,Information Engineering and Media,Yes,2021-06-07,NCS Applications,...,Agree,Yes,NaN,NaN,NaN,NaN,0,0,0,0
4,5,2021-08-06 11:42:27,2021-08-06 11:48:05,anonymous,NaN,Bachelor's,Business information system,No,2020-06-02,NCS Applications,...,Neutral,Yes,normal ok good,"nice, supportive efficent",kind okay,stressful complex,1,2,0,-2


In [132]:
words_pe = data.filter(['CLEANSE_PE'])

In [133]:
wnl = nltk.WordNetLemmatizer()
def lemmaNVAR(wpos):
    lemmas = []
    for w, pos in wpos:
        if pos[0] in ('N', 'V', 'J', 'R'):
            if pos in ('NNS', 'RBR', 'RBS', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):
                lemmas.append(wnl.lemmatize(w.lower(), pos = pos[0].lower()))
            elif pos in ('JJR', 'JJS'):
                lemmas.append(wnl.lemmatize(w.lower(), pos = 'a'))
            else:
                lemmas.append(w.lower())
    return lemmas


pe_words = list()
for line in words_pe['CLEANSE_PE']:
    if type(line) == float:
        continue
    else:
       for word in line.split():
          pe_words.append(word)  

pe_tokens = [word_tokenize(each_response) for each_response in pe_words]
pe_pos = [pos_tag(t) for t in pe_tokens]
pe_lem = [lemmaNVAR(t) for t in pe_pos]

In [134]:
stop = stopwords.words("English") + ["ncs", "' m"]
pe_tmp = []
for i in range(len(pe_lem)):
    pe_tmp = [word for word in pe_lem if word != []]

In [135]:
pe_tmp

[['varied'],
 ['full'],
 ['learning'],
 ['quite'],
 ['enrich'],
 ['new'],
 ['knowledge'],
 ['learn'],
 ['stressful'],
 ['complex'],
 ['old'],
 ['tech'],
 ['debugging'],
 ['tedious'],
 ['frustrate'],
 ['time'],
 ['not'],
 ['much'],
 ['useful'],
 ['career'],
 ['progression'],
 ['due'],
 ['lot'],
 ['repetitive'],
 ['work'],
 ['old'],
 ['code'],
 ['enrich'],
 ['learn'],
 ['okay'],
 ['fast'],
 ['pace'],
 ['reward'],
 ['stress'],
 ['tired'],
 ['over-worked'],
 ['enjoyable'],
 ['learnable'],
 ['deliverable'],
 ['eye-opening'],
 ['tedious'],
 ['real'],
 ['challenging'],
 ['fulfilling'],
 ['satisfied'],
 ['okay'],
 ['good'],
 ['learn'],
 ['fast'],
 ['pace'],
 ['fulfilling'],
 ['meaningful'],
 ['jade'],
 ['start'],
 ['stagnate'],
 ['largely'],
 ['positive'],
 ['work'],
 ['experience'],
 ['colleague'],
 ['knowledge'],
 ['erudition'],
 ['improvement'],
 ['plateau'],
 ['steep-learning-curve'],
 ['ot'],
 ['bad'],
 ['management'],
 ['bad'],
 ['time'],
 ['management'],
 ['bad'],
 ['experience'],
 ['no

In [136]:
pe_clean = []
for i in range(len(pe_tmp)):
    pe_tmp_clean = [ t for t in pe_tmp[i] if t not in stop]
    pe_clean.append(pe_tmp_clean)

In [137]:
# Use dictionary (built from corpus) to prepare a DTM (using frequency)
import logging
import gensim 
from gensim import corpora

In [138]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Filter off any words with document frequency less than 2, or appearing in more than 90% documents
dictionary = corpora.Dictionary(pe_clean)
print(dictionary)
dictionary.filter_extremes(no_below=2, no_above=0.8)

2021-10-23 16:52:33,295 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-10-23 16:52:33,302 : INFO : built Dictionary(195 unique tokens: ['varied', 'full', 'learning', 'quite', 'enrich']...) from 361 documents (total 342 corpus positions)
2021-10-23 16:52:33,305 : INFO : discarding 133 tokens: [('varied', 1), ('full', 1), ('complex', 1), ('debugging', 1), ('frustrate', 1), ('much', 1), ('career', 1), ('progression', 1), ('due', 1), ('code', 1)]...
2021-10-23 16:52:33,307 : INFO : keeping 62 tokens which were in no less than 2 and no more than 288 (=80.0%) documents
2021-10-23 16:52:33,309 : INFO : resulting dictionary: Dictionary(62 unique tokens: ['learning', 'quite', 'enrich', 'new', 'knowledge']...)


Dictionary(195 unique tokens: ['varied', 'full', 'learning', 'quite', 'enrich']...)


In [139]:
print(dictionary)

Dictionary(62 unique tokens: ['learning', 'quite', 'enrich', 'new', 'knowledge']...)


In [140]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
from gensim.models.coherencemodel import CoherenceModel

In [141]:
lda1 = gensim.models.ldamodel.LdaModel(dtm, num_topics = 10, 
                                      id2word = dictionary, 
                                      passes=10,chunksize=32,
                                      random_state=10)
cm_cv1 = CoherenceModel(lda1,  dictionary=dictionary, texts=pe_clean, coherence='c_v')

2021-10-23 16:52:37,220 : INFO : using symmetric alpha at 0.1
2021-10-23 16:52:37,222 : INFO : using symmetric eta at 0.1
2021-10-23 16:52:37,227 : INFO : using serial LDA version on this node
2021-10-23 16:52:37,230 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 361 documents, updating model once every 32 documents, evaluating perplexity every 320 documents, iterating 50x with a convergence threshold of 0.001000
2021-10-23 16:52:37,232 : INFO : PROGRESS: pass 0, at document #32/361
2021-10-23 16:52:37,248 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:37,253 : INFO : topic #5 (0.100): 0.396*"enrich" + 0.396*"learn" + 0.004*"old" + 0.003*"useful" + 0.003*"lot" + 0.003*"new" + 0.003*"stressful" + 0.003*"fast" + 0.003*"okay" + 0.003*"time"
2021-10-23 16:52:37,256 : INFO : topic #6 (0.100): 0.651*"pace" + 0.006*"enrich" + 0.006*"old" + 0.006*"learn" + 0.006*"lot" + 0.006*"work" + 0.006*"stressf

2021-10-23 16:52:37,377 : INFO : topic #0 (0.100): 0.263*"challenge" + 0.083*"tedious" + 0.053*"old" + 0.031*"enrich" + 0.031*"lot" + 0.009*"interesting" + 0.009*"ask" + 0.009*"manageable" + 0.009*"stagnant" + 0.009*"process"
2021-10-23 16:52:37,378 : INFO : topic diff=0.354821, rho=0.408248
2021-10-23 16:52:37,379 : INFO : PROGRESS: pass 0, at document #224/361
2021-10-23 16:52:37,387 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:37,389 : INFO : topic #5 (0.100): 0.738*"learn" + 0.010*"enrich" + 0.004*"fruitful" + 0.004*"grow" + 0.004*"interesting" + 0.004*"bad" + 0.004*"stagnant" + 0.004*"management" + 0.004*"ask" + 0.004*"manageable"
2021-10-23 16:52:37,392 : INFO : topic #2 (0.100): 0.441*"busy" + 0.213*"mundane" + 0.031*"work" + 0.005*"fruitful" + 0.005*"interesting" + 0.005*"stagnant" + 0.005*"grow" + 0.005*"ask" + 0.005*"manageable" + 0.005*"bad"
2021-10-23 16:52:37,393 : INFO : topic #6 (0.100): 0.468*"tire" + 0.185*"boring" + 0.053*"

2021-10-23 16:52:37,519 : INFO : topic #3 (0.100): 0.426*"learnt" + 0.145*"bad" + 0.109*"learning" + 0.047*"time" + 0.034*"interest" + 0.008*"management" + 0.004*"motivation" + 0.004*"lot" + 0.004*"quite" + 0.004*"fruitful"
2021-10-23 16:52:37,522 : INFO : topic #5 (0.100): 0.582*"learn" + 0.008*"enrich" + 0.007*"learnt" + 0.007*"motivation" + 0.007*"lot" + 0.007*"quite" + 0.007*"fruitful" + 0.007*"team" + 0.007*"system" + 0.007*"enriching"
2021-10-23 16:52:37,523 : INFO : topic diff=0.371237, rho=0.288675
2021-10-23 16:52:37,524 : INFO : PROGRESS: pass 1, at document #32/361
2021-10-23 16:52:37,536 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:37,539 : INFO : topic #5 (0.100): 0.505*"learn" + 0.251*"enrich" + 0.004*"learnt" + 0.004*"motivation" + 0.004*"lot" + 0.004*"quite" + 0.004*"fruitful" + 0.004*"old" + 0.004*"team" + 0.004*"system"
2021-10-23 16:52:37,541 : INFO : topic #9 (0.100): 0.227*"stagnant" + 0.182*"enriching" + 0.143*"new" + 0

2021-10-23 16:52:37,648 : INFO : topic #4 (0.100): 0.153*"ask" + 0.150*"manageable" + 0.084*"difficult" + 0.079*"limited" + 0.079*"opportunity" + 0.079*"process" + 0.078*"stuff" + 0.078*"progressive" + 0.050*"fast" + 0.003*"learnt"
2021-10-23 16:52:37,649 : INFO : topic diff=0.188800, rho=0.274398
2021-10-23 16:52:37,650 : INFO : PROGRESS: pass 1, at document #224/361
2021-10-23 16:52:37,662 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:37,666 : INFO : topic #3 (0.100): 0.205*"learning" + 0.182*"bad" + 0.178*"interest" + 0.078*"management" + 0.065*"time" + 0.051*"learnt" + 0.004*"motivation" + 0.004*"lot" + 0.004*"quite" + 0.004*"fruitful"
2021-10-23 16:52:37,667 : INFO : topic #7 (0.100): 0.253*"good" + 0.171*"experience" + 0.148*"challenging" + 0.072*"enjoyable" + 0.046*"reward" + 0.046*"meaningful" + 0.046*"eye-opening" + 0.046*"improvement" + 0.046*"fulfilling" + 0.015*"colleague"
2021-10-23 16:52:37,668 : INFO : topic #4 (0.100): 0.194*"

2021-10-23 16:52:37,787 : INFO : topic #0 (0.100): 0.386*"lot" + 0.175*"fruitful" + 0.088*"tedious" + 0.081*"grow" + 0.071*"challenge" + 0.009*"old" + 0.003*"enrich" + 0.003*"learnt" + 0.003*"motivation" + 0.003*"team"
2021-10-23 16:52:37,789 : INFO : topic #9 (0.100): 0.315*"stagnant" + 0.247*"enriching" + 0.090*"new" + 0.079*"system" + 0.070*"knowledge" + 0.050*"useful" + 0.003*"learnt" + 0.003*"motivation" + 0.003*"fruitful" + 0.003*"team"
2021-10-23 16:52:37,790 : INFO : topic diff=0.297703, rho=0.274398
2021-10-23 16:52:37,791 : INFO : PROGRESS: pass 2, at document #32/361
2021-10-23 16:52:37,803 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:37,806 : INFO : topic #1 (0.100): 0.371*"okay" + 0.183*"stressful" + 0.137*"interesting" + 0.122*"tech" + 0.003*"learn" + 0.003*"learnt" + 0.003*"motivation" + 0.003*"lot" + 0.003*"quite" + 0.003*"fruitful"
2021-10-23 16:52:37,807 : INFO : topic #4 (0.100): 0.188*"fast" + 0.091*"difficult" + 0.086*"a

...

2021-10-23 16:52:39,362 : INFO : topic #6 (0.100): 0.261*"tire" + 0.212*"boring" + 0.096*"overwhelm" + 0.032*"pace" + 0.007*"quite" + 0.007*"motivation" + 0.007*"lot" + 0.007*"colleague" + 0.007*"stagnant" + 0.007*"stress"
2021-10-23 16:52:39,363 : INFO : topic #0 (0.100): 0.347*"lot" + 0.189*"fruitful" + 0.090*"tedious" + 0.086*"grow" + 0.076*"challenge" + 0.015*"old" + 0.004*"motivation" + 0.004*"quite" + 0.004*"colleague" + 0.004*"stress"
2021-10-23 16:52:39,365 : INFO : topic #7 (0.100): 0.238*"good" + 0.149*"challenging" + 0.121*"experience" + 0.067*"colleague" + 0.053*"stress" + 0.039*"enjoyable" + 0.035*"learnable" + 0.035*"start" + 0.027*"fulfilling" + 0.024*"improvement"
2021-10-23 16:52:39,366 : INFO : topic diff=0.245573, rho=0.227736
2021-10-23 16:52:39,367 : INFO : PROGRESS: pass 8, at document #32/361
2021-10-23 16:52:39,378 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:39,381 : INFO : topic #5 (0.100): 0.533*"learn" + 0.220*"en

2021-10-23 16:52:39,495 : INFO : topic #3 (0.100): 0.259*"bad" + 0.115*"learnt" + 0.112*"learning" + 0.111*"management" + 0.099*"time" + 0.064*"interest" + 0.004*"meaningful" + 0.004*"smooth" + 0.004*"reward" + 0.004*"technical"
2021-10-23 16:52:39,497 : INFO : topic #1 (0.100): 0.244*"stressful" + 0.203*"okay" + 0.198*"interesting" + 0.055*"tech" + 0.005*"management" + 0.005*"old" + 0.005*"smooth" + 0.005*"challenge" + 0.005*"colleague" + 0.005*"stagnant"
2021-10-23 16:52:39,498 : INFO : topic diff=0.150509, rho=0.222051
2021-10-23 16:52:39,499 : INFO : PROGRESS: pass 8, at document #224/361
2021-10-23 16:52:39,509 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:39,511 : INFO : topic #7 (0.100): 0.243*"good" + 0.172*"experience" + 0.140*"challenging" + 0.074*"enjoyable" + 0.049*"fulfilling" + 0.045*"reward" + 0.045*"eye-opening" + 0.045*"meaningful" + 0.045*"improvement" + 0.019*"colleague"
2021-10-23 16:52:39,513 : INFO : topic #8 (0.100): 0.

2021-10-23 16:52:39,626 : INFO : topic #7 (0.100): 0.238*"good" + 0.148*"challenging" + 0.122*"experience" + 0.065*"colleague" + 0.053*"stress" + 0.040*"enjoyable" + 0.035*"learnable" + 0.035*"start" + 0.027*"fulfilling" + 0.025*"eye-opening"
2021-10-23 16:52:39,629 : INFO : topic #6 (0.100): 0.262*"tire" + 0.211*"boring" + 0.097*"overwhelm" + 0.033*"pace" + 0.007*"lot" + 0.007*"quite" + 0.007*"colleague" + 0.007*"motivation" + 0.007*"okay" + 0.007*"stagnant"
2021-10-23 16:52:39,630 : INFO : topic #4 (0.100): 0.118*"ask" + 0.113*"difficult" + 0.092*"manageable" + 0.080*"process" + 0.080*"limited" + 0.080*"opportunity" + 0.068*"progressive" + 0.068*"stuff" + 0.025*"fast" + 0.005*"lot"
2021-10-23 16:52:39,631 : INFO : topic diff=0.239382, rho=0.222051
2021-10-23 16:52:39,632 : INFO : PROGRESS: pass 9, at document #32/361
2021-10-23 16:52:39,640 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:39,642 : INFO : topic #4 (0.100): 0.152*"fast" + 0.095*

2021-10-23 16:52:39,748 : INFO : topic #9 (0.100): 0.244*"stagnant" + 0.155*"new" + 0.116*"knowledge" + 0.104*"enriching" + 0.062*"useful" + 0.038*"system" + 0.005*"management" + 0.005*"improvement" + 0.005*"old" + 0.005*"reward"
2021-10-23 16:52:39,749 : INFO : topic #4 (0.100): 0.148*"ask" + 0.144*"manageable" + 0.083*"difficult" + 0.078*"limited" + 0.078*"process" + 0.078*"opportunity" + 0.077*"progressive" + 0.077*"stuff" + 0.061*"fast" + 0.003*"management"
2021-10-23 16:52:39,750 : INFO : topic #1 (0.100): 0.245*"stressful" + 0.205*"okay" + 0.196*"interesting" + 0.056*"tech" + 0.005*"old" + 0.005*"management" + 0.005*"smooth" + 0.005*"eye-opening" + 0.005*"technical" + 0.005*"meaningful"
2021-10-23 16:52:39,751 : INFO : topic diff=0.146838, rho=0.216771
2021-10-23 16:52:39,752 : INFO : PROGRESS: pass 9, at document #224/361
2021-10-23 16:52:39,763 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:52:39,765 : INFO : topic #4 (0.100): 0.182*"mana

2021-10-23 16:52:39,894 : INFO : topic #4 (0.100): 0.118*"ask" + 0.112*"difficult" + 0.093*"manageable" + 0.080*"process" + 0.080*"limited" + 0.080*"opportunity" + 0.069*"stuff" + 0.069*"progressive" + 0.026*"fast" + 0.005*"lot"
2021-10-23 16:52:39,896 : INFO : topic #7 (0.100): 0.238*"good" + 0.148*"challenging" + 0.123*"experience" + 0.064*"colleague" + 0.052*"stress" + 0.041*"enjoyable" + 0.035*"learnable" + 0.035*"start" + 0.028*"fulfilling" + 0.025*"improvement"
2021-10-23 16:52:39,897 : INFO : topic #0 (0.100): 0.338*"lot" + 0.192*"fruitful" + 0.090*"tedious" + 0.087*"grow" + 0.077*"challenge" + 0.016*"old" + 0.004*"colleague" + 0.004*"motivation" + 0.004*"stress" + 0.004*"quite"
2021-10-23 16:52:39,898 : INFO : topic #6 (0.100): 0.264*"tire" + 0.210*"boring" + 0.098*"overwhelm" + 0.034*"pace" + 0.007*"motivation" + 0.007*"quite" + 0.007*"colleague" + 0.007*"lot" + 0.007*"stress" + 0.007*"okay"
2021-10-23 16:52:39,899 : INFO : topic diff=0.233638, rho=0.216771


In [142]:
lda_cv1 = cm_cv1.get_coherence()
print(lda_cv1)

2021-10-23 16:52:40,437 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate probabilities from sliding windows
2021-10-23 16:52:44,214 : INFO : 7 accumulators retrieved from output queue
2021-10-23 16:52:44,236 : INFO : accumulated word occurrence stats for 209 virtual documents


0.8513730932752857


In [143]:
print(lda1.log_perplexity(dtm))

2021-10-23 16:52:46,508 : INFO : -5.448 per-word bound, 43.7 perplexity estimate based on a held-out corpus of 361 documents with 209 words


-5.448431337506458


In [144]:
vis1 = pyLDAvis.gensim.prepare(lda1, dtm, dictionary)
vis1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.054472  0.055663       1        1  17.200928
8     -0.337713 -0.049704       2        1  10.502392
4      0.015058  0.010248       3        1   9.784687
5      0.010104  0.005765       4        1   9.545460
3      0.025532  0.022081       5        1   9.545456
9      0.113736 -0.290852       6        1   9.306230
2      0.057578  0.195592       7        1   8.827750
0      0.027623  0.024636       8        1   8.827746
1      0.023434  0.020629       9        1   8.588529
6      0.010177  0.005942      10        1   7.870821, topic_info=           Term       Freq      Total Category  logprob  loglift
5         learn  13.000000  13.000000  Default    30.00  30.0000
61       learnt   9.000000   9.000000  Default    29.00  29.0000
15         okay   7.000000   7.000000  Default    28.00  28.0000
60         team   7.000000   7.000000  Default    27.00  27.0000
12          lot   6.000000   6.000000  Default    26.00  26.0000
..          ...        ...        ...      ...      ...      ...
53     stagnant   0.111949   6.286758  Topic10    -4.99  -1.4861
13   repetitive   0.111949   6.540737  Topic10    -4.99  -1.5257
61       learnt   0.111949   9.003680  Topic10    -4.99  -1.8453
9       tedious   0.111949   2.408738  Topic10    -4.99  -0.5268
23  challenging   0.111949   6.031999  Topic10    -4.99  -1.4448

[472 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
48        3  0.641481          ask
31        5  0.796005          bad
39       10  0.722031       boring
40        7  0.865481         busy
52        8  0.462180    challenge
23        1  0.828913  challenging
29        1  0.660902    colleague
49        3  0.669310    difficult
20        1  0.458227    enjoyable
2         4  0.815447       enrich
57        6  0.901979    enriching
36        2  0.491927     exciting
28        1  0.776215   experience
22        1  0.617475  eye-opening
16        3  0.809079         fast
55        8  0.932711     fruitful
24        1  0.583923   fulfilling
34        2  0.641602          fun
25        1  0.972307         good
33        2  0.828150        great
56        8  0.848349         grow
30        1  0.617475  improvement
37        5  0.634142     interest
54        9  0.887846  interesting
4         6  0.461262    knowledge
5         4  0.920645        learn
21        1  0.502596    learnable
0         5  0.698778     learning
61        5  0.888526       learnt
51        3  0.855563      limited
12        8  0.859291          lot
46        3  0.766730   manageable
26        1  0.617475   meaningful
59        7  0.750752   motivation
43        7  0.869774      mundane
3         6  0.748410          new
15        9  0.956693         okay
45        3  0.855563  opportunity
41       10  0.866045    overwhelm
17       10  0.793923         pace
47        3  0.855563      process
50        3  0.473039  progressive
1         2  0.806666        quite
13        2  0.917328   repetitive
18        1  0.617475       reward
35        2  0.757870       smooth
53        6  0.954387     stagnant
27        1  0.502596        start
19        1  0.771141       stress
6         9  0.836689    stressful
44        3  0.473039        stuff
58        6  0.835765       system
60        7  0.844186         team
8         9  0.628781         tech
9         8  0.830310      tedious
10        5  0.537966         time
42       10  0.794178         tire
11        6  0.556454       useful, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 5, 6, 4, 10, 3, 1, 2, 7])

In [145]:
# choosing num_topics = 4, since a number of clusters are situated in one area (suggesting that 4 topics would be enough for the dataset)
lda2 = gensim.models.ldamodel.LdaModel(dtm, num_topics = 4, 
                                      id2word = dictionary, 
                                      passes=10,chunksize=32,
                                      random_state=10)

2021-10-23 16:53:45,452 : INFO : using symmetric alpha at 0.25
2021-10-23 16:53:45,454 : INFO : using symmetric eta at 0.25
2021-10-23 16:53:45,457 : INFO : using serial LDA version on this node
2021-10-23 16:53:45,460 : INFO : running online (multi-pass) LDA training, 4 topics, 10 passes over the supplied corpus of 361 documents, updating model once every 32 documents, evaluating perplexity every 320 documents, iterating 50x with a convergence threshold of 0.001000
2021-10-23 16:53:45,461 : INFO : PROGRESS: pass 0, at document #32/361
2021-10-23 16:53:45,484 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:53:45,486 : INFO : topic #0 (0.250): 0.230*"old" + 0.226*"enrich" + 0.119*"repetitive" + 0.117*"time" + 0.115*"learning" + 0.007*"learn" + 0.006*"quite" + 0.006*"okay" + 0.006*"tedious" + 0.006*"fast"
2021-10-23 16:53:45,488 : INFO : topic #1 (0.250): 0.176*"learn" + 0.089*"tech" + 0.088*"stressful" + 0.087*"work" + 0.087*"lot" + 0.086*"knowledg

2021-10-23 16:53:45,629 : INFO : topic diff=0.340509, rho=0.377964
2021-10-23 16:53:45,630 : INFO : PROGRESS: pass 0, at document #256/361
2021-10-23 16:53:45,642 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:53:45,644 : INFO : topic #0 (0.250): 0.164*"fun" + 0.101*"great" + 0.089*"time" + 0.066*"learning" + 0.064*"interest" + 0.047*"exciting" + 0.047*"smooth" + 0.042*"bad" + 0.041*"repetitive" + 0.037*"interesting"
2021-10-23 16:53:45,646 : INFO : topic #1 (0.250): 0.249*"learn" + 0.194*"fruitful" + 0.156*"stressful" + 0.068*"tech" + 0.046*"grow" + 0.032*"stagnant" + 0.009*"work" + 0.009*"knowledge" + 0.009*"tedious" + 0.009*"okay"
2021-10-23 16:53:45,649 : INFO : topic #2 (0.250): 0.235*"good" + 0.112*"experience" + 0.106*"challenging" + 0.063*"learnable" + 0.063*"start" + 0.055*"new" + 0.045*"enjoyable" + 0.033*"reward" + 0.033*"improvement" + 0.031*"meaningful"
2021-10-23 16:53:45,651 : INFO : topic #3 (0.250): 0.093*"tire" + 0.076*"busy" + 

2021-10-23 16:53:45,819 : INFO : topic diff=0.299836, rho=0.274398
2021-10-23 16:53:45,821 : INFO : PROGRESS: pass 1, at document #96/361
2021-10-23 16:53:45,831 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:53:45,833 : INFO : topic #0 (0.250): 0.127*"bad" + 0.075*"repetitive" + 0.075*"management" + 0.069*"learning" + 0.062*"time" + 0.061*"great" + 0.053*"fun" + 0.047*"exciting" + 0.046*"enriching" + 0.042*"interest"
2021-10-23 16:53:45,835 : INFO : topic #1 (0.250): 0.238*"learn" + 0.111*"okay" + 0.080*"lot" + 0.066*"knowledge" + 0.066*"tedious" + 0.063*"stagnant" + 0.053*"work" + 0.042*"stressful" + 0.033*"useful" + 0.029*"tech"
2021-10-23 16:53:45,835 : INFO : topic #2 (0.250): 0.152*"experience" + 0.135*"good" + 0.105*"enjoyable" + 0.065*"learnt" + 0.057*"fulfilling" + 0.050*"challenging" + 0.049*"pace" + 0.040*"colleague" + 0.037*"new" + 0.034*"learnable"
2021-10-23 16:53:45,837 : INFO : topic #3 (0.250): 0.125*"quite" + 0.109*"team" + 0.08

2021-10-23 16:53:45,964 : INFO : PROGRESS: pass 1, at document #320/361
2021-10-23 16:53:45,976 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:53:45,978 : INFO : topic #0 (0.250): 0.127*"enriching" + 0.100*"bad" + 0.093*"interesting" + 0.093*"fun" + 0.080*"system" + 0.072*"learning" + 0.071*"great" + 0.052*"exciting" + 0.034*"time" + 0.025*"interest"
2021-10-23 16:53:45,979 : INFO : topic #1 (0.250): 0.228*"learn" + 0.121*"stressful" + 0.098*"fruitful" + 0.080*"grow" + 0.062*"useful" + 0.059*"motivation" + 0.036*"tech" + 0.028*"stagnant" + 0.020*"okay" + 0.015*"lot"
2021-10-23 16:53:45,981 : INFO : topic #2 (0.250): 0.191*"good" + 0.170*"challenging" + 0.124*"experience" + 0.040*"new" + 0.039*"learnable" + 0.039*"start" + 0.037*"enjoyable" + 0.025*"reward" + 0.025*"improvement" + 0.025*"meaningful"
2021-10-23 16:53:45,983 : INFO : topic #3 (0.250): 0.158*"busy" + 0.084*"tire" + 0.060*"mundane" + 0.052*"challenge" + 0.052*"difficult" + 0.049*"team

2021-10-23 16:53:46,149 : INFO : topic #0 (0.250): 0.129*"bad" + 0.081*"great" + 0.079*"technical" + 0.076*"fun" + 0.058*"repetitive" + 0.056*"management" + 0.052*"learning" + 0.047*"time" + 0.036*"exciting" + 0.035*"enriching"
2021-10-23 16:53:46,151 : INFO : topic #1 (0.250): 0.334*"learn" + 0.097*"stressful" + 0.073*"okay" + 0.055*"lot" + 0.044*"knowledge" + 0.044*"tedious" + 0.043*"stagnant" + 0.036*"work" + 0.023*"useful" + 0.021*"fruitful"
2021-10-23 16:53:46,152 : INFO : topic #2 (0.250): 0.160*"good" + 0.159*"experience" + 0.095*"challenging" + 0.080*"fulfilling" + 0.068*"enjoyable" + 0.056*"new" + 0.044*"learnt" + 0.033*"pace" + 0.027*"colleague" + 0.023*"start"
2021-10-23 16:53:46,154 : INFO : topic #3 (0.250): 0.082*"ask" + 0.081*"manageable" + 0.076*"tire" + 0.054*"busy" + 0.046*"challenge" + 0.046*"difficult" + 0.044*"quite" + 0.043*"opportunity" + 0.043*"limited" + 0.043*"process"
2021-10-23 16:53:46,155 : INFO : topic diff=0.314105, rho=0.264616
2021-10-23 16:53:46,156 :

...

2021-10-23 16:53:47,376 : INFO : topic diff=0.211509, rho=0.227736
2021-10-23 16:53:47,377 : INFO : PROGRESS: pass 7, at document #64/361
2021-10-23 16:53:47,389 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:53:47,391 : INFO : topic #0 (0.250): 0.144*"repetitive" + 0.092*"enriching" + 0.074*"enrich" + 0.074*"old" + 0.065*"learning" + 0.060*"interesting" + 0.053*"great" + 0.051*"time" + 0.044*"bad" + 0.043*"fun"
2021-10-23 16:53:47,393 : INFO : topic #1 (0.250): 0.151*"learn" + 0.129*"okay" + 0.097*"lot" + 0.076*"stagnant" + 0.075*"tedious" + 0.075*"knowledge" + 0.059*"work" + 0.055*"stressful" + 0.040*"fruitful" + 0.039*"useful"
2021-10-23 16:53:47,394 : INFO : topic #2 (0.250): 0.100*"good" + 0.094*"learnt" + 0.076*"challenging" + 0.075*"fulfilling" + 0.068*"experience" + 0.064*"pace" + 0.054*"new" + 0.054*"colleague" + 0.046*"enjoyable" + 0.045*"start"
2021-10-23 16:53:47,395 : INFO : topic #3 (0.250): 0.129*"quite" + 0.111*"team" + 0.085*"bus

2021-10-23 16:53:47,515 : INFO : topic #0 (0.250): 0.130*"fun" + 0.098*"great" + 0.083*"bad" + 0.071*"exciting" + 0.068*"interesting" + 0.055*"enriching" + 0.048*"time" + 0.046*"system" + 0.044*"learning" + 0.042*"repetitive"
2021-10-23 16:53:47,516 : INFO : topic #1 (0.250): 0.290*"learn" + 0.145*"stressful" + 0.119*"fruitful" + 0.043*"tech" + 0.040*"stagnant" + 0.033*"grow" + 0.032*"okay" + 0.025*"lot" + 0.020*"knowledge" + 0.020*"tedious"
2021-10-23 16:53:47,517 : INFO : topic #2 (0.250): 0.212*"good" + 0.141*"experience" + 0.123*"challenging" + 0.047*"new" + 0.046*"enjoyable" + 0.041*"start" + 0.041*"learnable" + 0.031*"fulfilling" + 0.029*"reward" + 0.029*"improvement"
2021-10-23 16:53:47,518 : INFO : topic #3 (0.250): 0.080*"tire" + 0.078*"ask" + 0.073*"busy" + 0.061*"manageable" + 0.053*"opportunity" + 0.053*"limited" + 0.053*"process" + 0.049*"overwhelm" + 0.045*"progressive" + 0.045*"stuff"
2021-10-23 16:53:47,519 : INFO : topic diff=0.147492, rho=0.227736
2021-10-23 16:53:47,

2021-10-23 16:53:47,687 : INFO : topic #0 (0.250): 0.134*"bad" + 0.087*"great" + 0.082*"fun" + 0.069*"repetitive" + 0.059*"learning" + 0.058*"management" + 0.052*"time" + 0.045*"enriching" + 0.039*"exciting" + 0.036*"old"
2021-10-23 16:53:47,689 : INFO : topic #1 (0.250): 0.305*"learn" + 0.102*"stressful" + 0.085*"okay" + 0.064*"lot" + 0.051*"stagnant" + 0.049*"tedious" + 0.049*"knowledge" + 0.040*"work" + 0.028*"fruitful" + 0.027*"useful"
2021-10-23 16:53:47,691 : INFO : topic #2 (0.250): 0.173*"experience" + 0.138*"good" + 0.106*"challenging" + 0.075*"enjoyable" + 0.063*"new" + 0.055*"learnt" + 0.044*"fulfilling" + 0.038*"pace" + 0.032*"colleague" + 0.027*"start"
2021-10-23 16:53:47,693 : INFO : topic #3 (0.250): 0.103*"busy" + 0.087*"quite" + 0.076*"team" + 0.072*"tire" + 0.067*"boring" + 0.064*"mundane" + 0.057*"overwhelm" + 0.054*"fast" + 0.042*"stress" + 0.036*"eye-opening"
2021-10-23 16:53:47,694 : INFO : topic diff=0.125749, rho=0.222051
2021-10-23 16:53:47,696 : INFO : PROGRES

2021-10-23 16:53:47,864 : INFO : topic #1 (0.250): 0.234*"learn" + 0.122*"fruitful" + 0.095*"stressful" + 0.059*"tedious" + 0.059*"knowledge" + 0.057*"grow" + 0.044*"useful" + 0.030*"tech" + 0.028*"stagnant" + 0.023*"okay"
2021-10-23 16:53:47,866 : INFO : topic #2 (0.250): 0.191*"good" + 0.120*"challenging" + 0.099*"experience" + 0.077*"new" + 0.060*"learnt" + 0.054*"colleague" + 0.034*"enjoyable" + 0.030*"learnable" + 0.030*"start" + 0.024*"fulfilling"
2021-10-23 16:53:47,867 : INFO : topic #3 (0.250): 0.160*"busy" + 0.072*"tire" + 0.062*"motivation" + 0.058*"boring" + 0.050*"mundane" + 0.043*"ask" + 0.041*"team" + 0.041*"difficult" + 0.041*"challenge" + 0.037*"stress"
2021-10-23 16:53:47,868 : INFO : topic diff=0.117269, rho=0.222051
2021-10-23 16:53:47,874 : INFO : -5.701 per-word bound, 52.0 perplexity estimate based on a held-out corpus of 9 documents with 7 words
2021-10-23 16:53:47,876 : INFO : PROGRESS: pass 8, at document #361/361
2021-10-23 16:53:47,880 : INFO : merging chang

2021-10-23 16:53:47,992 : INFO : topic #2 (0.250): 0.179*"good" + 0.145*"experience" + 0.082*"enjoyable" + 0.070*"challenging" + 0.055*"fulfilling" + 0.049*"reward" + 0.049*"improvement" + 0.049*"meaningful" + 0.042*"new" + 0.038*"learnt"
2021-10-23 16:53:47,993 : INFO : topic #3 (0.250): 0.091*"tire" + 0.081*"busy" + 0.064*"boring" + 0.063*"mundane" + 0.060*"ask" + 0.058*"manageable" + 0.050*"eye-opening" + 0.042*"quite" + 0.036*"team" + 0.035*"difficult"
2021-10-23 16:53:47,994 : INFO : topic diff=0.132633, rho=0.216771
2021-10-23 16:53:47,995 : INFO : PROGRESS: pass 9, at document #224/361
2021-10-23 16:53:48,007 : INFO : merging changes from 32 documents into a model of 361 documents
2021-10-23 16:53:48,009 : INFO : topic #0 (0.250): 0.152*"fun" + 0.076*"bad" + 0.072*"learning" + 0.070*"repetitive" + 0.059*"interest" + 0.053*"exciting" + 0.050*"smooth" + 0.050*"great" + 0.049*"interesting" + 0.043*"technical"
2021-10-23 16:53:48,010 : INFO : topic #1 (0.250): 0.281*"learn" + 0.135*

In [146]:
cm_cv2 = CoherenceModel(lda2,  dictionary=dictionary, texts=pe_clean, coherence='c_v')

In [147]:
lda_cv2 = cm_cv2.get_coherence()
print(lda_cv2)

2021-10-23 16:53:52,829 : INFO : using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate probabilities from sliding windows
2021-10-23 16:53:56,977 : INFO : 7 accumulators retrieved from output queue
2021-10-23 16:53:56,987 : INFO : accumulated word occurrence stats for 209 virtual documents


0.85320488089106


In [148]:
print(lda2.log_perplexity(dtm))

2021-10-23 16:54:01,310 : INFO : -5.089 per-word bound, 34.0 perplexity estimate based on a held-out corpus of 361 documents with 209 words


-5.089488366475274


In [116]:
vis2 = pyLDAvis.gensim.prepare(lda2, dtm, dictionary)
vis2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.063110 -0.023845       1        1  25.660060
0      0.088074 -0.204072       2        1  25.177181
3      0.118287  0.208623       3        1  24.937986
1     -0.269471  0.019294       4        1  24.224773, topic_info=           Term       Freq      Total Category  logprob  loglift
61       learnt  11.000000  11.000000  Default  30.0000  30.0000
13   repetitive   8.000000   8.000000  Default  29.0000  29.0000
25         good   8.000000   8.000000  Default  28.0000  28.0000
60         team   8.000000   8.000000  Default  27.0000  27.0000
5         learn   7.000000   7.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
54  interesting   0.170294   5.150992   Topic4  -5.6948  -1.9916
40         busy   0.170293   6.627140   Topic4  -5.6948  -2.2436
21    learnable   0.170292   1.875671   Topic4  -5.6948  -0.9814
27        start   0.170292   1.875671   Topic4  -5.6948  -0.9814
46   manageable   0.170287   1.939633   Topic4  -5.6948  -1.0150

[206 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
48        3  0.880930          ask
31        2  0.744833          bad
39        3  0.706913       boring
40        3  0.905368         busy
52        3  0.914828    challenge
23        1  0.883013  challenging
29        1  0.702706    colleague
49        3  0.914828    difficult
20        1  0.972608    enjoyable
57        2  0.924271    enriching
36        2  0.806624     exciting
28        1  0.826567   experience
22        3  0.747473  eye-opening
55        4  0.916962     fruitful
24        1  0.618724   fulfilling
34        2  0.750786          fun
25        1  0.921355         good
33        2  0.850625        great
56        4  0.844388         grow
30        1  0.654001  improvement
37        2  0.619312     interest
54        2  0.970687  interesting
4         4  0.825930    knowledge
5         4  0.895647        learn
21        1  0.533142    learnable
0         2  0.660158     learning
61        1  0.989329       learnt
51        3  0.566969      limited
12        4  0.979764          lot
46        3  0.515561   manageable
32        2  0.811059   management
26        1  0.654001   meaningful
59        3  0.676250   motivation
43        3  0.791599      mundane
3         1  0.781214          new
15        4  0.959379         okay
45        3  0.566969  opportunity
41        3  0.590300    overwhelm
17        1  0.876098         pace
47        3  0.566969      process
50        3  0.617839  progressive
1         3  0.968703        quite
13        2  0.910864   repetitive
18        1  0.654001       reward
35        2  0.674771       smooth
53        4  0.942145     stagnant
27        1  0.533142        start
19        3  0.495214       stress
6         4  0.844119    stressful
44        3  0.617839        stuff
58        2  0.656848       system
60        3  0.976077         team
8         4  0.643752         tech
38        2  0.725654    technical
9         4  0.825930      tedious
10        2  0.519460         time
42        3  0.889849         tire
11        4  0.507441       useful, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])